##신입기수 교육: RNN Assignment
: text generation using RNN

: 데이터 이해/전처리를 읽고 난 후 모델 및 Text Generation 함수를 설계해보기

: 참고) text generation using RNN.ipynb파일, https://wikidocs.net/45101


**1) 데이터 이해/전처리**

In [ ]:
import pandas as pd
from string import punctuation
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ArticlesApril2018.csv to ArticlesApril2018 (6).csv


In [ ]:
# ArticlesApril2018.csv 파일 로드
df = pd.read_csv('ArticlesApril2018.csv')

In [ ]:
print('Number of columns: ', len(df.columns))
print(df.columns)

Number of columns:  15
Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')


In [ ]:
df['headline'].isnull().values.any()

False

In [ ]:
# 모든 기사의 제목 하나의 리스트로 저장
headline = []
headline.extend(list(df.headline.values))
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'Unknown',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?']

In [ ]:
# headline 중 unknown 처리

print('총 샘플의 개수 : {}'.format(len(headline))) # 현재 샘플의 개수
headline = [n for n in headline if n != "Unknown"] # Unknown 값을 가진 샘플 제거
print('노이즈값 제거 후 샘플의 개수 : {}'.format(len(headline))) # 제거 후 샘플의 개수

총 샘플의 개수 : 1324
노이즈값 제거 후 샘플의 개수 : 1214


In [ ]:
headline[:10]

['Former N.F.L. Cheerleaders’ Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'How a Bag of Texas Dirt  Became a Times Tradition',
 'Is School a Place for Self-Expression?',
 'Commuter Reprogramming',
 'Ford Changed Leaders, Looking for a Lift. It’s Still Looking.',
 'Romney Failed to Win at Utah Convention, But Few Believe He’s Doomed',
 'Chain Reaction',
 'He Forced the Vatican to Investigate Sex Abuse. Now He’s Meeting With Pope Francis.']

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re

# 데이터 전처리: 구두점 제거, lowercase()
def data_preprocessing(text):
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower()
    return text

cnt=0
for i in headline:
    headline[cnt] = data_preprocessing(i)
    cnt+=1
headline[:10]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained',
 'how a bag of texas dirt  became a times tradition',
 'is school a place for selfexpression',
 'commuter reprogramming',
 'ford changed leaders looking for a lift its still looking',
 'romney failed to win at utah convention but few believe hes doomed',
 'chain reaction',
 'he forced the vatican to investigate sex abuse now hes meeting with pope francis']

In [ ]:
# 토큰화 및 단어 집합 크기 확인
t= Tokenizer()
t.fit_on_texts(headline)
vocab_size = len(t.word_index) + 1

print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 3494


In [ ]:
# 정수 인코딩 & 하나의 문장을 2 이상 길이의 훈련 데이터로 만들기"
sequences = []
for text in headline:
    for line in text.split('\n'):    # \n 기준으로 문장 토큰화
        encoded = t.texts_to_sequences([line])[0]
        for i in range(1, len(encoded)):
            sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
            sequences.append(sequence)

cnt = 0
for i in sequences:
    if cnt < 11:
        cnt+=1
        print(i)

[99, 269]
[99, 269, 371]
[99, 269, 371, 1115]
[99, 269, 371, 1115, 582]
[99, 269, 371, 1115, 582, 52]
[99, 269, 371, 1115, 582, 52, 7]
[99, 269, 371, 1115, 582, 52, 7, 2]
[99, 269, 371, 1115, 582, 52, 7, 2, 372]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10]
[99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116]
[100, 3]


In [ ]:
print(sequences)

[[99, 269], [99, 269, 371], [99, 269, 371, 1115], [99, 269, 371, 1115, 582], [99, 269, 371, 1115, 582, 52], [99, 269, 371, 1115, 582, 52, 7], [99, 269, 371, 1115, 582, 52, 7, 2], [99, 269, 371, 1115, 582, 52, 7, 2, 372], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10], [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116], [100, 3], [100, 3, 1117], [100, 3, 1117, 2], [100, 3, 1117, 2, 14], [100, 3, 1117, 2, 14, 583], [100, 3, 1117, 2, 14, 583, 24], [100, 3, 1117, 2, 14, 583, 24, 1118], [100, 3, 1117, 2, 14, 583, 24, 1118, 373], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5], [100, 3, 1117, 2, 14, 583, 24, 1118, 373, 374, 5, 1119], [1, 14], [1, 14, 1120], [1, 14, 1120, 1121], [15, 2], [15, 2, 584], [15, 2, 584, 4], [15, 2, 584, 4, 215], [15, 2, 584, 4, 215, 375], [15, 2, 584, 4, 215, 375, 1122], [15, 2, 584, 4, 215, 375, 1122, 2], [15, 2, 584, 4, 215, 375, 1122, 2, 376], [15, 2, 584, 4, 215, 375, 1122, 2, 376, 585], [8, 167], [8, 167, 2], [8,

In [ ]:
# 빈도수 상위 단어 확인
index_to_word={}
for key, value in t.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 576번 단어 : {}'.format(index_to_word[576]))
print('빈도수 상위 1번 단어 : {}'.format(index_to_word[1]))

빈도수 상위 576번 단어 : victims
빈도수 상위 1번 단어 : the


In [ ]:
# 가장 긴 샘플의 길이에 전체 샘플 길이 맞추기
max_len = max(len(l) for l in sequences) 
print('샘플 최대 길이 : {}'.format(max_len))

# 전체 샘플 길이를 max_len으로 패딩 작업
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

print()
print('패딩 결과 확인')
print(sequences)

샘플 최대 길이 : 24

패딩 결과 확인
[[   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 [   0    0    0 ...  269  371 1115]
 ...
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]
 [   0    0    0 ...  115    2 1025]]


In [ ]:
# 훈련 데이터에서 레이블 분리 (각 문장의 마지막 단어만 분리해서 y로)
sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

print('X 확인')
print(X)
print()
print('y 확인')
print(y)

# 레이블에 대한 one-hot encoding
y = to_categorical(y, num_classes = vocab_size)

print()
print('one-hot encoding : y 확인')
print(y)

X 확인
[[   0    0    0 ...    0    0   99]
 [   0    0    0 ...    0   99  269]
 [   0    0    0 ...   99  269  371]
 ...
 [   0    0    0 ...  170    8 3493]
 [   0    0    0 ...    8 3493  115]
 [   0    0    0 ... 3493  115    2]]

y 확인
[ 269  371 1115 ...  115    2 1025]

one-hot encoding : y 확인
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


**2) 모델 설계하기**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam

In [ ]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model = Sequential()

model.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model.add(SimpleRNN(256))   # 모델 및 은닉 상태 크기 설정  
model.add(Dense(vocab_size, activation='softmax')) #Dense Layer 및 활성화 함수 설정
Dropout(0.5)
# adam = Adam(learning_rate=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정
model.summary()
model.fit(X,y,epochs = 70, verbose=1) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 -> epochs 1000으로 해도 93%를 넘지 않아요...
'''
  처음에 다운 받은 실습 파일에서 60% 이상이 과제인 줄 알고 있다가 2월 9일에 공지사항 올라온 것 보고 95%까지 도전해봤지만 실패했습니다.
  여러 조합으로 해봤는데 92.5% 정도가 최대로 나왔습니다.
  이 정도로 다시 나올 거라 생각했는데, 최고로 높은 accuracy를 보인 모델은 날라갔습니다...
'''
# model.fit(X,y,epochs = 100, verbose=1, validation_split=0.1) -> val_loss 계속 증가 -> overfitting? 

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 23, 32)            111808    
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 256)               73984     
                                                                 
 dense_4 (Dense)             (None, 3494)              897958    
                                                                 
Total params: 1,083,750
Trainable params: 1,083,750
Non-trainable params: 0
_________________________________________________________________
Epoch 1/70
244/244 [==============================] - 10s 35ms/step - loss: 7.7078 - accuracy: 0.0245
Epoch 2/70
244/244 [==============================] - 10s 40ms/step - loss: 7.1648 - accuracy: 0.0304
Epoch 3/70
244/244 [==============================] - 9s 38ms/step - loss: 6.8865 - accuracy: 0.0461
Epo

In [ ]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model1 = Sequential()

model1.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model1.add(SimpleRNN(128))   # 모델 및 은닉 상태 크기 설정  
model1.add(Dense(vocab_size/2, activation='relu')) #Dense Layer 및 활성화 함수 설정
model1.add(Dropout(0.3))
model1.add(Dense(vocab_size, activation='softmax'))


# adam = Adam(learning_rate=0.01)
model1.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정
model1.fit(X,y,epochs = 70, verbose=1) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 -> epochs 1000으로 해도 93%를 넘지 않아요... 92.5% 정도가 최대로 나왔습니다...
# model.fit(X,y,epochs = 100, verbose=1, validation_split=0.1) -> val_loss 계속 증가 -> overfitting? 

Epoch 1/70
244/244 [==============================] - 28s 104ms/step - loss: 7.4200 - accuracy: 0.0268
Epoch 2/70
244/244 [==============================] - 24s 100ms/step - loss: 6.9578 - accuracy: 0.0428
Epoch 3/70
244/244 [==============================] - 23s 94ms/step - loss: 6.3598 - accuracy: 0.0557
Epoch 4/70
244/244 [==============================] - 27s 109ms/step - loss: 5.7231 - accuracy: 0.0714
Epoch 5/70
244/244 [==============================] - 36s 148ms/step - loss: 4.9591 - accuracy: 0.1178
Epoch 6/70
244/244 [==============================] - 25s 104ms/step - loss: 3.8821 - accuracy: 0.2411
Epoch 7/70
244/244 [==============================] - 23s 96ms/step - loss: 2.7021 - accuracy: 0.4327
Epoch 8/70
244/244 [==============================] - 23s 96ms/step - loss: 1.8179 - accuracy: 0.6099
Epoch 9/70
244/244 [==============================] - 24s 97ms/step - loss: 1.2472 - accuracy: 0.7266
Epoch 10/70
244/244 [==============================] - 23s 95ms/step - loss: 

In [ ]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model2 = Sequential()

model2.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model2.add(SimpleRNN(64))   # 모델 및 은닉 상태 크기 설정  
model2.add(Dense(vocab_size, activation='softmax')) #Dense Layer 및 활성화 함수 설정

# adam = Adam(learning_rate=0.01)
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정
model2.fit(X,y,epochs = 200, verbose=1, batch_size = 128) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 -> epochs 1000으로 해도 93%를 넘지 않아요... 92.5% 정도가 최대로 나왔습니다...
# model.fit(X,y,epochs = 100, verbose=1, validation_split=0.1) -> val_loss 계속 증가 -> overfitting? 

Epoch 1/200
61/61 [==============================] - 4s 35ms/step - loss: 7.7353 - accuracy: 0.0249
Epoch 2/200
61/61 [==============================] - 2s 28ms/step - loss: 7.1419 - accuracy: 0.0313
Epoch 3/200
61/61 [==============================] - 2s 30ms/step - loss: 7.0898 - accuracy: 0.0313
Epoch 4/200
61/61 [==============================] - 2s 28ms/step - loss: 7.0156 - accuracy: 0.0334
Epoch 5/200
61/61 [==============================] - 2s 31ms/step - loss: 6.9133 - accuracy: 0.0436
Epoch 6/200
61/61 [==============================] - 2s 30ms/step - loss: 6.7951 - accuracy: 0.0483
Epoch 7/200
61/61 [==============================] - 2s 28ms/step - loss: 6.7513 - accuracy: 0.0511
Epoch 8/200
61/61 [==============================] - 2s 31ms/step - loss: 7.0909 - accuracy: 0.0441
Epoch 9/200
61/61 [==============================] - 2s 31ms/step - loss: 6.5666 - accuracy: 0.0510
Epoch 10/200
61/61 [==============================] - 2s 30ms/step - loss: 6.4385 - accuracy: 0.0543

In [ ]:
model2.fit(X,y,epochs = 200, verbose=1, batch_size = 128)

In [ ]:
# FILL IN 파트 - RNN 모델 설계 직접해보기 
model2 = Sequential()

model2.add(Embedding(vocab_size, 32, input_length = max_len-1)) # 각 단어를 32차원으로 embedding, 레이블 분리 후 길이는 -1
model2.add(SimpleRNN(64))   # 모델 및 은닉 상태 크기 설정  
model2.add(Dense(vocab_size, activation='relu')) #Dense Layer 및 활성화 함수 설정
model2.add(Dropout(0.5))
model2.add(Dense(vocab_size, activation='softmax'))
# adam = Adam(learning_rate=0.01)
model2.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy']) # Loss, Optimizer, Metrics 설정
model2.fit(X,y,epochs = 200, verbose=1, batch_size = 128) #최종 Accuracy가 최소 95% 이상은 나오게 epochs 설정 -> epochs 1000으로 해도 93%를 넘지 않아요... 92.5% 정도가 최대로 나왔습니다...
# model.fit(X,y,epochs = 100, verbose=1, validation_split=0.1) -> val_loss 계속 증가 -> overfitting? 

Epoch 1/200
61/61 [==============================] - 24s 355ms/step - loss: 7.5085 - accuracy: 0.0264
Epoch 2/200
61/61 [==============================] - 22s 354ms/step - loss: 7.0769 - accuracy: 0.0299
Epoch 3/200
61/61 [==============================] - 22s 359ms/step - loss: 6.7857 - accuracy: 0.0406
Epoch 4/200
61/61 [==============================] - 21s 344ms/step - loss: 6.3583 - accuracy: 0.0473
Epoch 5/200
61/61 [==============================] - 21s 341ms/step - loss: 5.7568 - accuracy: 0.0636
Epoch 6/200
61/61 [==============================] - 21s 342ms/step - loss: 5.0641 - accuracy: 0.1016
Epoch 7/200
61/61 [==============================] - 21s 350ms/step - loss: 4.2979 - accuracy: 0.1817
Epoch 8/200
61/61 [==============================] - 21s 345ms/step - loss: 3.5955 - accuracy: 0.2837
Epoch 9/200
61/61 [==============================] - 22s 353ms/step - loss: 2.9781 - accuracy: 0.3773
Epoch 10/200
61/61 [==============================] - 21s 346ms/step - loss: 2.485

In [1]:
# 문장 생성 함수
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre') # 데이터에 대한 패딩
        predict_x=model.predict(encoded, verbose=2) 
        classes_x=np.argmax(predict_x,axis=1)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == classes_x: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [ ]:
# Text Generation 결과 확인 
# 임의의 단어 'good'에 대해서 10개의 단어를 추가 생성
print(sentence_generation(model, t, 'good', 10))

1/1 - 0s - 193ms/epoch - 193ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
1/1 - 0s - 19ms/epoch - 19ms/step
1/1 - 0s - 28ms/epoch - 28ms/step
1/1 - 0s - 20ms/epoch - 20ms/step
good riddance mr speaker ponsot influential and care lash video on
